# Mask Notebook
- Demonstrates how we will apply a mask over the data
- Should grab the data that is labeled as either 0 or 1, not 9999
- Apply the mask to these data

In [1]:
# Import libraries
import pickle
from utils import *
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#open pickle file
path = "data/data_path_dictionary.pkl"
data_path_dictionary = open_pickle(path)

print("Dictionary contents: ")
data_path_dictionary

Dictionary contents: 


defaultdict(list,
            {'subject_data': ['svm_data/10004_08693/svm_subj_vecs.mat',
              'svm_data/10008_09924/svm_subj_vecs.mat',
              'svm_data/10009_08848/svm_subj_vecs.mat',
              'svm_data/10016_09694/svm_subj_vecs.mat',
              'svm_data/10017_08894/svm_subj_vecs.mat',
              'svm_data/10018_08907/svm_subj_vecs.mat',
              'svm_data/10021_08839/svm_subj_vecs.mat',
              'svm_data/10022_08854/svm_subj_vecs.mat',
              'svm_data/10023_09126/svm_subj_vecs.mat',
              'svm_data/10027_09455/svm_subj_vecs.mat',
              'svm_data/10033_08871/svm_subj_vecs.mat',
              'svm_data/10034_08879/svm_subj_vecs.mat',
              'svm_data/10035_08847/svm_subj_vecs.mat',
              'svm_data/10036_09800/svm_subj_vecs.mat',
              'svm_data/10037_09903/svm_subj_vecs.mat',
              'svm_data/10038_09063/svm_subj_vecs.mat',
              'svm_data/10039_08941/svm_subj_vecs.mat',
              

In [3]:
# single subject data paths from S3
masks_path = data_path_dictionary['mask_data'][0]
subject_path = data_path_dictionary['subject_data'][0]
label_path = data_path_dictionary['labels'][0]


In [4]:
%%time
# Get mat data from aws
mask_data = access_load_data(masks_path)
mask_data.keys()


CPU times: user 157 ms, sys: 43.1 ms, total: 200 ms
Wall time: 787 ms


dict_keys(['__header__', '__version__', '__globals__', 'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'])

In [9]:
%%time
# Get label data from aws
label_data = access_load_data(label_path)
label_data.keys()

CPU times: user 75.8 ms, sys: 6.89 ms, total: 82.7 ms
Wall time: 461 ms


dict_keys(['__header__', '__version__', '__globals__', 'decrease_onsets', 'increase_onsets', 'rest_ons', 'rt_labels'])

In [5]:
# Takes in filepaths and mask type and returns a numpy-compatible mask
def create_mask(mask_data_filepath, mask_type='mask'):
    mask_type_dict = access_load_data(mask_data_filepath)
    np_array_mask = mask_type_dict[mask_type]
    np_compatible_mask = np.ma.make_mask(np_array_mask).reshape(79*95*79)
    return np_compatible_mask


# create_mask example
mask_data_filepath = 'svm_data/masks/masks.mat'
mask_type = 'masksubACC'

sub_acc_mask = create_mask(mask_data_filepath, mask_type)
sub_acc_mask

array([False, False, False, ..., False, False, False])

In [6]:
# Takes in a user dictionary of four runs and a mask. Returns a user dictionary with mask type and four masked runs
def mask_user(subject, mask):
    masked_runs = {'mask_type':mask}
    for i in range(4):
        user_key = 'run_0' + str(i+1) + '_vec'
        array = subject[user_key]
        array = array[:, mask] ## Will need to also get the subject data that is labeled either 0,1
        masked_runs[user_key]=array
    return masked_runs


# example of mask_user using sub_acc_mask above
subject_path = 'svm_data/10004_08693/svm_subj_vecs.mat'
first_subject = access_load_data(subject_path)
mask_for_function = sub_acc_mask #From cell above

masked_first_subject = mask_user(first_subject, mask_for_function)
masked_first_subject

{'mask_type': array([False, False, False, ..., False, False, False]),
 'run_01_vec': array([[   0,    0,    0, ..., 3428, 3954, 4105],
        [   0,    0,    0, ..., 3318, 3747, 3905],
        [   0,    0,    0, ..., 3359, 3721, 3876],
        ...,
        [   0,    0,    0, ..., 3327, 3783, 3940],
        [   0,    0,    0, ..., 3319, 3912, 4011],
        [   0,    0,    0, ..., 3305, 3905, 4022]], dtype=int16),
 'run_02_vec': array([[   0,    0,    0, ..., 3719, 3986, 4713],
        [   0,    0,    0, ..., 3576, 3814, 4568],
        [   0,    0,    0, ..., 3595, 3682, 4327],
        ...,
        [   0,    0,    0, ..., 3691, 4094, 4855],
        [   0,    0,    0, ..., 3594, 3854, 4482],
        [   0,    0,    0, ..., 3583, 3808, 4607]], dtype=int16),
 'run_03_vec': array([[   0,    0,    0, ..., 3726, 4493, 5430],
        [   0,    0,    0, ..., 3589, 4246, 5278],
        [   0,    0,    0, ..., 3568, 4112, 5008],
        ...,
        [   0,    0,    0, ..., 3393, 3948, 4800],
   

In [7]:
masked_first_subject['run_01_vec'].shape

(144, 235266)